In [85]:
import pandas as pd
from elftools.elf.elffile import ELFFile

In [86]:
config = "fdsc_inter-rasm"
old_csv = "~/faults_" + config + "_new.csv"
new_csv = "faults_" + config + "_names.csv"
bin_name = "../out_" + config + "_1.elf"

data = pd.read_csv(old_csv)
data

,attempt,stop_addr,stop_fn,delay,target,bitflip,code,seed
0,0,0x08003f90,vTaskSwitchContext,0.495,0x200040f6,32,0,3735928559
1,1,0x08018b0a,vTaskDelay_dup,0.704,0x200013f8,32,0,3735928559
2,2,0x0801d0d8,crc32buf_ret_dup,0.717,0x200058b2,16,0,3735928559
3,3,0x0802170e,xTimerCreateStatic_ret_dup,0.915,0x20000717,32,0,3735928559
4,4,0x0801d0d8,crc32buf_ret_dup,0.379,0x200002aa,8,0,3735928559
...,...,...,...,...,...,...,...,...
59995,59995,0x08012e24,vListInitialise_dup,0.033,0x20004e75,8,0,3735928559
59996,59996,0x08002f08,xTaskIncrementTick,0.830,0x20003ca3,1,0,3735928559
59997,59997,0x08019eec,Multiply_dup,0.242,0x20002d90,16,0,3735928559
59998,59998,0x0801f74a,vQueueDelete_dup,0.379,0x200060f1,128,0,3735928559


In [87]:
symtable = dict()

with open(bin_name, 'rb') as elffile:
    for section in ELFFile(elffile).iter_sections():
        try:
            for sym in section.iter_symbols():
                if (sym.name != "ucHeap" and sym.entry['st_info']['type'] == 'STT_OBJECT'):
                    symtable[sym.name] = {"addr": sym.entry['st_value'], "size": sym.entry['st_size']}
        except: 
            continue

In [88]:
data_size = data.shape[0]

new_data = [None for i in range(data_size)]

def associate_symbols():

    for idx, row in data.iterrows():
        found = False
        for sym in symtable.keys():
            try:
                if int(row['target'][2:], base=16) in range(symtable[sym]['addr'], symtable[sym]['addr']+symtable[sym]['size'], 1):
                    #print(symtable[sym])
                    new_data[idx] = sym
                    break
            except:
                break
        print(str(idx) + "/" + str(data_size), end = "\r")

associate_symbols()

data['struct'] = new_data

In [89]:
data.to_csv(new_csv)

In [90]:
'''
    37: 20001900  2000 OBJECT  LOCAL  DEFAULT   10 main.xBufferRecvStack
    34: 20001068  2000 OBJECT  LOCAL  DEFAULT   10 main.xTaskStack
'''

print(hex(0x20001900 + 2000))
print(hex(0x20001068 + 2000))

0x200020d0
0x20001838
